In [1]:
import torch
torch.__version__

'1.13.0.dev20220706+cu113'

In [2]:
# import FSDP and BackwardPrefetch class
from torch.distributed.fsdp import (
    FullyShardedDataParallel as FSDP,
    BackwardPrefetch,
    StateDictType,
    FullStateDictConfig,  # general model non-sharded, non-flattened params
    LocalStateDictConfig,  # flattened params, usable only by FSDP
    # ShardedStateDictConfig, # un-flattened param but shards, usable by other parallel schemes.
)

backward_prefetch param in FSDP init can take three states:

1 - None (i.e. don't pass anything) = summon for next FSDP unit comes after the all_reduce for current layer gradients

2 - BackwardPrefetch.BACKWARD_POST = prefetch at the end of current FSDP unit computation, after params are dropped

3 - BackwardPrefetch.BACKWARD_PRE = prefetch at start of current FSDP unit computation (earliest, adds to peak memory)

In [ ]:
prefetch_policy = BackwardPrefetch.BACKWARD_PRE  # BackwardPrefetch.BACKWARD_POST or None

# ----- main FSDP init -----------
    model = FSDP(
        model,
        auto_wrap_policy=my_auto_wrap_policy,
        mixed_precision=mp_policy,
        backward_prefetch=prefetch_policy,
        device_id=torch.cuda.current_device(),
        sharding_strategy=ShardingStrategy.FULL_SHARD,  # Zero2
        forward_prefetch=True,
    )

In [1]:
# basic testing - 
# PRE can improve speed up to 13% with .59% increased peak memory

BackwardPrefetch.BACKWARD_PRE


Animations:

![Backwards_START](images/backwardPass_start.png)

![Backwards_all_gather](images/all_gather_inmotion.png)

PRE:
![Backwards_PRE](images/backward_pre.png)

POST:![Backwards_NONE](images/post_bpf.png)

None: 
![Backwards_NONE](images/none_bpf.png)
